In [8]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the dataset
df = pd.read_csv('cleaned_Amazon_Sale_Report.csv', encoding='ISO-8859-1', low_memory=False)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month_name()

month_order = ['January', 'February','March', 'April', 'May', 
               'June', 'July', 'August', 'September',
               'October', 'November', 'December']

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Sales Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Div([
            html.Label('State'),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in df['ship-state'].unique()],
                value=None,
                clearable=True,
                style={'width': '100%'}
            ),
        ], className='dropdown-container', style={'width': '24%', 'display': 'inline-block'}),

        html.Div([
            html.Label('Category'),
            dcc.Dropdown(
                id='category-dropdown',
                options=[{'label': category, 'value': category} for category in df['Category'].unique()],
                value=None,
                clearable=True,
                style={'width': '100%'}
            ),
        ], className='dropdown-container', style={'width': '24%', 'display': 'inline-block'}),

        html.Div([
            html.Label('SKU'),
            dcc.Dropdown(
                id='sku-dropdown',
                options=[{'label': sku, 'value': sku} for sku in df['SKU'].unique()],
                value=None,
                clearable=True,
                style={'width': '100%'}
            ),
        ], className='dropdown-container', style={'width': '24%', 'display': 'inline-block'}),

        html.Div([
            html.Label('Month'),
            dcc.Dropdown(
                id='month-dropdown',
                options=[{'label': month, 'value': month} for month in month_order],
                value=None,
                clearable=True,
                style={'width': '100%'}
            ),
        ], className='dropdown-container', style={'width': '24%', 'display': 'inline-block'}),
    ], className='dropdowns', style={'marginBottom': '20px'}),

    html.Div([
        dcc.Graph(id='sales-distribution-by-state', className='graph'),
        dcc.Graph(id='monthly-sales-trend', className='graph'),
        dcc.Graph(id='sales-by-category-barh', className='graph'),
    ], className='graphs'),

    # Key Metrics
    html.Div([
        html.Div([
            html.H4("Total Sales"),
            html.P(f"${df['Amount'].sum():,.2f}")
        ], className='metric'),
        html.Div([
            html.H4("Total Orders"),
            html.P(f"{df['Order ID'].nunique():,}")
        ], className='metric'),
        html.Div([
            html.H4("Average Order Value"),
            html.P(f"${df['Amount'].mean():,.2f}")
        ], className='metric'),
        html.Div([
            html.H4("Total Quantity Sold"),
            html.P(f"{df['Qty'].sum():,}")
        ], className='metric'),
    ], className='metrics-container', style={'display': 'flex', 'justify-content': 'space-around', 'margin': '20px'}),

    # Sales Trend over Time
    html.Div([
        dcc.Graph(id='sales-trend-line-chart')
    ], className='chart-container'),

    # Sales by Region
    html.Div([
        dcc.Graph(id='sales-by-region-map')
    ], className='chart-container'),

    # Hidden Interval component to trigger callback
    dcc.Interval(id='interval-component', interval=1*1000, n_intervals=0)
], className='container')

# Define callbacks to update graphs based on dropdown selections
@app.callback(
    Output('sales-distribution-by-state', 'figure'),
    Output('monthly-sales-trend', 'figure'),
    Output('sales-by-category-barh', 'figure'),
    Output('sales-trend-line-chart', 'figure'),
    Output('sales-by-region-map', 'figure'),
    Input('state-dropdown', 'value'),
    Input('category-dropdown', 'value'),
    Input('sku-dropdown', 'value'),
    Input('month-dropdown', 'value')
)
def update_plots(state, category, sku, month):
    filtered_df = df.copy()

    if state:
        filtered_df = filtered_df[filtered_df['ship-state'] == state]
    if category:
        filtered_df = filtered_df[filtered_df['Category'] == category]
    if sku:
        filtered_df = filtered_df[filtered_df['SKU'] == sku]
    if month:
        filtered_df = filtered_df[filtered_df['Month'] == month]

    # Plot Sales Distribution by State
    state_sales = filtered_df.groupby('ship-city')['Amount'].sum().sort_values(ascending=False)[:10]
    fig_sales_distribution_by_state = px.bar(
        x=state_sales.index,
        y=state_sales.values,
        labels={'x': 'City', 'y': 'Total Sales Amount'},
        title=f'Top 10 Cities in {state} by Sales Amount'
    )

    # Plot Monthly Sales Trend
    monthly_sales_trend = filtered_df.groupby('Month')['Amount'].sum().reset_index()
    monthly_sales_trend['Month'] = pd.Categorical(monthly_sales_trend['Month'], categories=month_order, ordered=True)
    fig_monthly_sales_trend = px.line(
        monthly_sales_trend,
        x='Month',
        y='Amount',
        labels={'x': 'Month', 'y': 'Sales Amount'},
        title='Monthly Sales Trend'
    )

    # Plot Sales Distribution by Category
    category_sales = filtered_df.groupby('Category')['Amount'].sum().sort_values(ascending=False)
    fig_sales_by_category_barh = px.bar(
        x=category_sales.values,
        y=category_sales.index,
        labels={'x': 'Total Sales Amount', 'y': 'Category'},
        title='Sales Distribution by Category',
        orientation='h'
    )

    # Sales Trend Line Chart
    sales_trend = df.groupby('Date')['Amount'].sum().reset_index()
    fig_sales_trend_line_chart = px.line(
        sales_trend,
        x='Date',
        y='Amount',
        title='Sales Trend Over Time'
    )

    # Sales by Region Map
    sales_by_region = df.groupby('ship-state')['Amount'].sum().reset_index()
    fig_sales_by_region_map = px.choropleth(
        sales_by_region,
        locations='ship-state',
        locationmode='USA-states',
        color='Amount',
        scope='usa',
        title='Sales by Region'
    )

    return fig_sales_distribution_by_state, fig_monthly_sales_trend, fig_sales_by_category_barh, \
           fig_sales_trend_line_chart, fig_sales_by_region_map

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
